In [1]:
import statsmodels.api as sm


def smooth(y):
    #return savitzky_golay(y, window_size=2001, order=3)
    return lowess(y)

# 0.2가 제일 잘 없앴음
def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T

import numpy as np
import pandas as pd
import os, pickle, sys
from scipy import signal
#sys.path.append('DL_model')


### input 설정
SRATE = 250 # 250Hz
LEN_INPUT = 20 # input 10s
LEN_PER_NRS = 60 # vital length for each NRS
OVERLAP = 2
n_aug = int((LEN_PER_NRS-LEN_INPUT)/OVERLAP) + 1 # data augmentation 개수




input_path = '../DL_model/dataset/preprocess4/input3/'
if not os.path.exists(input_path[:-1]):
    os.mkdir(input_path[:-1])

if os.path.exists(input_path+'x_train_pacu.npz'):
    print('loading train...', flush=True, end='')
    x_train_pacu = np.load(input_path+'x_train_pacu.npz', allow_pickle=True)['arr_0']
    y_train_pacu = np.load(input_path+'y_train_pacu.npz')['arr_0']
    x_test_pacu = np.load(input_path+'x_test_pacu.npz', allow_pickle=True)['arr_0']
    y_test_pacu = np.load(input_path+'y_test_pacu.npz')['arr_0']
    print('done', flush=True)

else:
    false_row_list_preop = []
    
    vital_path = '../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_2min/'
    ecg_path = '../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_2min/ECG,'
    df_preprocess_pacu = pickle.load(open('../DL_model/cache/preprocess3/input3/df_preprocess_pacu_agender','rb'))
    df_demograph = pd.read_csv('../DL_model/caseids_age_gender.csv')
    

    ### test set에 해당하는 file_path
    caseid_test = pickle.load(open('../DL_model/caseid_test_new', 'rb'))
    caseid_train = pickle.load(open('../DL_model/caseid_train_new', 'rb'))
    caseid_val = pickle.load(open('../DL_model/caseid_val_new', 'rb'))
    
    non_lis = []
    x_train_pacu, y_train_pacu = [], []
    x_test_pacu, y_test_pacu = [], []
    x_val_pacu, y_val_pacu = [], []
    age_train_pacu, gender_train_pacu = [], []
    age_test_pacu, gender_test_pacu = [], []
    age_val_pacu, gender_val_pacu = [], []

    cnt = 0
    for _, row in df_preprocess_pacu.iterrows():
        cnt += 1
        if cnt<=3600:
            continue
        if cnt>5400:
            break
            
            
        print('loading data {}/{} ...'.format(cnt, len(df_preprocess_pacu)), end='')
        row_demo = df_demograph[df_demograph['caseids']==row['caseids']].iloc[0]
            

        # vital data - PPG (resampling 100 Hz to 250 Hz)
        df_vital = pickle.load(open(vital_path+row['file_path'], 'rb')).reset_index()
        pleth_samp = df_vital[['Pleth']].fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        pleth_resamp = signal.resample(pleth_samp, 120*SRATE)
        ppg_per_NRS = np.full(30000, np.nan)
        ppg_per_NRS[0:len(pleth_resamp)] = pleth_resamp
        

        # vital data - ECG (250Hz)
        ecg_samp = pickle.load(open(ecg_path+row['file_path'][:-3]+'vital', 'rb')).reset_index()[['ECG']]
        ecg_samp = ecg_samp.fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()[0:30000]
        ecg_per_NRS = np.full(30000,np.nan)
        ecg_per_NRS[0:len(ecg_samp)] = ecg_samp
   

        save_path = '../../cranberry2/Preprocessing/cache/lowess_filtered/pacu_'+row['caseids']
        # 한 NRS에 대해 23개의 input 확인
        for i in range(0,30):
            # input이 전처리 통과한 경우
            if row[str(i+1)]:
                start_idx = i*OVERLAP*SRATE # 500i
                end_idx = (i*OVERLAP+LEN_INPUT)*SRATE # 500i + 1000
                
                # input의 normalization
                ppg_inp = np.copy(ppg_per_NRS[start_idx:end_idx])
                if np.sum(np.isnan(ppg_inp))!=0:
                    ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                
                pleth_inp = ppg_inp - smooth(ppg_inp)
                pleth_inp = pleth_inp - np.nanmean(pleth_inp)

                ecg_inp = np.copy(ecg_per_NRS[start_idx:end_idx])
                if np.sum(np.isnan(ecg_inp))!=0:
                    ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                
                ecg_inp = ecg_inp - smooth(ecg_inp)
                ecg_inp = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)
                
                
                pickle.dump([pleth_inp, ecg_inp], open(save_path+'_{}'.format(i), 'wb'))
                # 해당 caseid가 test set에 속하는 경우
                if row['caseids'] in caseid_test:
                    age_test_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_test_pacu.append(1)
                    else:
                        gender_test_pacu.append(0)
                    x_test_pacu.append([pleth_inp, ecg_inp])
                    y_test_pacu.append(int(float(row['NRS'])))

                # 해당 caseid가 train set에 해당하는 경우
                elif row['caseids'] in caseid_val:
                    age_val_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_val_pacu.append(1)
                    else:
                        gender_val_pacu.append(0)                    
                    x_val_pacu.append([pleth_inp, ecg_inp])
                    y_val_pacu.append(int(float(row['NRS'])))
                    
                elif row['caseids'] in caseid_train:
                    age_train_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_train_pacu.append(1)
                    else:
                        gender_train_pacu.append(0)                    
                    x_train_pacu.append([pleth_inp, ecg_inp])
                    y_train_pacu.append(int(float(row['NRS'])))
                    
                else:
                    non_lis.append(row['caseids'])
                    
        print('completed')

    x_train_pacu = np.array(x_train_pacu, np.float32)
    x_test_pacu = np.array(x_test_pacu, np.float32)
    y_train_pacu = np.array(y_train_pacu, int)
    y_test_pacu = np.array(y_test_pacu, int)
    x_val_pacu = np.array(x_val_pacu, np.float32)
    y_val_pacu = np.array(y_val_pacu, int)
    
    age_train_pacu = np.array(age_train_pacu, int)
    age_test_pacu = np.array(age_test_pacu, int)
    age_val_pacu = np.array(age_val_pacu, int)
    gender_train_pacu = np.array(gender_train_pacu, int)
    gender_test_pacu = np.array(gender_test_pacu, int)
    gender_val_pacu = np.array(gender_val_pacu, int)
    
        
    # 저장하기
    print('saving...', end='', flush=True)
    np.savez_compressed(input_path+'x_train_pacu3.npz', x_train_pacu)
    np.savez_compressed(input_path+'x_test_pacu3.npz', x_test_pacu)
    np.savez_compressed(input_path+'x_val_pacu3.npz', x_val_pacu)
    np.savez_compressed(input_path+'y_train_pacu3.npz', y_train_pacu)
    np.savez_compressed(input_path+'y_test_pacu3.npz', y_test_pacu)
    np.savez_compressed(input_path+'y_val_pacu3.npz', y_val_pacu)
    
    np.savez_compressed(input_path+'age_train_pacu3.npz', age_train_pacu)
    np.savez_compressed(input_path+'age_test_pacu3.npz', age_test_pacu)
    np.savez_compressed(input_path+'age_val_pacu3.npz', age_val_pacu)    
    np.savez_compressed(input_path+'gender_train_pacu3.npz', gender_train_pacu)
    np.savez_compressed(input_path+'gender_test_pacu3.npz', gender_test_pacu)
    np.savez_compressed(input_path+'gender_val_pacu3.npz', gender_val_pacu)    
    
    print('done', flush=True)

    
    
    
print('size of training set(pacu):', len(x_train_pacu))
print('size of validation set(pacu):', len(x_val_pacu))
print('size of test set(pacu):', len(x_test_pacu))

loading data 3601/7253 ...completed
loading data 3602/7253 ...completed
loading data 3603/7253 ...completed
loading data 3604/7253 ...completed
loading data 3605/7253 ...completed
loading data 3606/7253 ...completed
loading data 3607/7253 ...completed
loading data 3608/7253 ...completed
loading data 3609/7253 ...completed
loading data 3610/7253 ...completed
loading data 3611/7253 ...completed
loading data 3612/7253 ...completed
loading data 3613/7253 ...completed
loading data 3614/7253 ...completed
loading data 3615/7253 ...completed
loading data 3616/7253 ...completed
loading data 3617/7253 ...completed
loading data 3618/7253 ...completed
loading data 3619/7253 ...completed
loading data 3620/7253 ...completed
loading data 3621/7253 ...completed
loading data 3622/7253 ...completed
loading data 3623/7253 ...completed
loading data 3624/7253 ...completed
loading data 3625/7253 ...completed
loading data 3626/7253 ...completed
loading data 3627/7253 ...completed
loading data 3628/7253 ...co

loading data 4055/7253 ...completed
loading data 4056/7253 ...completed
loading data 4057/7253 ...completed
loading data 4058/7253 ...completed
loading data 4059/7253 ...completed
loading data 4060/7253 ...completed
loading data 4061/7253 ...completed
loading data 4062/7253 ...completed
loading data 4063/7253 ...completed
loading data 4064/7253 ...completed
loading data 4065/7253 ...completed
loading data 4066/7253 ...completed
loading data 4067/7253 ...completed
loading data 4068/7253 ...completed
loading data 4069/7253 ...completed
loading data 4070/7253 ...completed
loading data 4071/7253 ...completed
loading data 4072/7253 ...completed
loading data 4073/7253 ...completed
loading data 4074/7253 ...completed
loading data 4075/7253 ...completed
loading data 4076/7253 ...completed
loading data 4077/7253 ...completed
loading data 4078/7253 ...completed
loading data 4079/7253 ...completed
loading data 4080/7253 ...completed
loading data 4081/7253 ...completed
loading data 4082/7253 ...co

loading data 4509/7253 ...completed
loading data 4510/7253 ...completed
loading data 4511/7253 ...completed
loading data 4512/7253 ...completed
loading data 4513/7253 ...completed
loading data 4514/7253 ...completed
loading data 4515/7253 ...completed
loading data 4516/7253 ...completed
loading data 4517/7253 ...completed
loading data 4518/7253 ...completed
loading data 4519/7253 ...completed
loading data 4520/7253 ...completed
loading data 4521/7253 ...completed
loading data 4522/7253 ...completed
loading data 4523/7253 ...completed
loading data 4524/7253 ...completed
loading data 4525/7253 ...completed
loading data 4526/7253 ...completed
loading data 4527/7253 ...completed
loading data 4528/7253 ...completed
loading data 4529/7253 ...completed
loading data 4530/7253 ...completed
loading data 4531/7253 ...completed
loading data 4532/7253 ...completed
loading data 4533/7253 ...completed
loading data 4534/7253 ...completed
loading data 4535/7253 ...completed
loading data 4536/7253 ...co

loading data 4964/7253 ...completed
loading data 4965/7253 ...completed
loading data 4966/7253 ...completed
loading data 4967/7253 ...completed
loading data 4968/7253 ...completed
loading data 4969/7253 ...completed
loading data 4970/7253 ...completed
loading data 4971/7253 ...completed
loading data 4972/7253 ...completed
loading data 4973/7253 ...completed
loading data 4974/7253 ...completed
loading data 4975/7253 ...completed
loading data 4976/7253 ...completed
loading data 4977/7253 ...completed
loading data 4978/7253 ...completed
loading data 4979/7253 ...completed
loading data 4980/7253 ...completed
loading data 4981/7253 ...completed
loading data 4982/7253 ...completed
loading data 4983/7253 ...completed
loading data 4984/7253 ...completed
loading data 4985/7253 ...completed
loading data 4986/7253 ...completed
loading data 4987/7253 ...completed
loading data 4988/7253 ...completed
loading data 4989/7253 ...completed
loading data 4990/7253 ...completed
loading data 4991/7253 ...co

loading data 5192/7253 ...completed
loading data 5193/7253 ...completed
loading data 5194/7253 ...completed
loading data 5195/7253 ...completed
loading data 5196/7253 ...completed
loading data 5197/7253 ...completed
loading data 5198/7253 ...completed
loading data 5199/7253 ...completed
loading data 5200/7253 ...completed
loading data 5201/7253 ...completed
loading data 5202/7253 ...completed
loading data 5203/7253 ...completed
loading data 5204/7253 ...completed
loading data 5205/7253 ...completed
loading data 5206/7253 ...completed
loading data 5207/7253 ...completed
loading data 5208/7253 ...completed
loading data 5209/7253 ...completed
loading data 5210/7253 ...completed
loading data 5211/7253 ...completed
loading data 5212/7253 ...completed
loading data 5213/7253 ...completed
loading data 5214/7253 ...completed
loading data 5215/7253 ...completed
loading data 5216/7253 ...completed
loading data 5217/7253 ...completed
loading data 5218/7253 ...completed
loading data 5219/7253 ...co

# preprocess5/lowess filter

In [5]:
vital_list = pickle.load(open('../Preprocessing/201227_modified_vital','rb'))
vital_list

['3.0,5654,PACU1_3_200604_180441.vital',
 '5.0,2038,PACU1_3_190902_160501.vital',
 '5.0,1883,PACU1_7_190827_182001.vital',
 '4.0,8963,PACU1_2_190828_105401.vital',
 '7.0,8990,PACU1_4_190902_133811.vital',
 '6.0,1404,PACU1_3_190720_113459.vital',
 '8.0,3907,PACU1_11_200220_154810.vital',
 '4.0,2241,PACU1_3_190906_181103.vital',
 '5.0,9038,PACU1_11_190909_134919.vital',
 '4.0,2123,PACU1_5_190904_154406.vital',
 '5.0,2968,PACU1_9_200113_160255.vital',
 '7.0,1587,PACU1_12_190820_104238.vital',
 '4.0,9003,PACU1_5_190904_135714.vital',
 '4.0,6475,PACU1_11_200715_145822.vital',
 '7.0,888,PACU1_5_190617_164932.vital',
 '7.0,1753,PACU1_4_190821_183002.vital',
 '5.0,2458,PACU1_1_191018_132325.vital',
 '3.0,8916,PACU1_4_190821_144052.vital',
 '5.0,2339,PACU1_7_190911_092247.vital',
 '8.0,3081,PACU1_5_200116_181354.vital',
 '8.0,8049,PACU1_9_201016_111800.vital',
 '3.0,2291,PACU1_2_190910_100401.vital',
 '2.0,2374,PACU1_12_190916_101642.vital',
 '5.0,2502,PACU1_10_191022_103919.vital',
 '3.0,803,P

In [6]:
import statsmodels.api as sm


def smooth(y):
    #return savitzky_golay(y, window_size=2001, order=3)
    return lowess(y)

# 0.2가 제일 잘 없앴음
def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T

import numpy as np
import pandas as pd
import os, pickle, sys
from scipy import signal
#sys.path.append('DL_model')


### input 설정
SRATE = 250 # 250Hz
LEN_INPUT = 20 # input 10s
LEN_PER_NRS = 300 # vital length for each NRS
OVERLAP = 5
n_aug = int((LEN_PER_NRS-LEN_INPUT)/OVERLAP) + 1 # data augmentation 개수


vital_list = pickle.load(open('../Preprocessing/201227_modified_vital','rb'))
input_path = '../DL_model/dataset/preprocess5/'
if not os.path.exists(input_path[:-1]):
    os.mkdir(input_path[:-1])

if os.path.exists(input_path+'x_train_pacu.npz'):
    print('loading train...', flush=True, end='')
    x_train_pacu = np.load(input_path+'x_train_pacu.npz', allow_pickle=True)['arr_0']
    y_train_pacu = np.load(input_path+'y_train_pacu.npz')['arr_0']
    x_test_pacu = np.load(input_path+'x_test_pacu.npz', allow_pickle=True)['arr_0']
    y_test_pacu = np.load(input_path+'y_test_pacu.npz')['arr_0']
    print('done', flush=True)

else:
    false_row_list_preop = []
    
    vital_path = '../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'
    ecg_path = '../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'
    df_preprocess_pacu = pickle.load(open('../DL_model/cache/preprocess5/df_preprocess_pacu_agender','rb'))
    df_demograph = pickle.load(open('../DL_model/df_caseids+age+gender', 'rb'))
    

    ### test set에 해당하는 file_path
    caseid_test = pickle.load(open('../DL_model/caseid_test_new', 'rb'))
    caseid_train = pickle.load(open('../DL_model/caseid_train_new', 'rb'))
    caseid_val = pickle.load(open('../DL_model/caseid_val_new', 'rb'))
    
    
    non_lis = []
    x_train_pacu, y_train_pacu = [], []
    x_test_pacu, y_test_pacu = [], []
    x_val_pacu, y_val_pacu = [], []
    age_train_pacu, gender_train_pacu = [], []
    age_test_pacu, gender_test_pacu = [], []
    age_val_pacu, gender_val_pacu = [], []

    
    cnt = 0
    for _, row in df_preprocess_pacu.iterrows():
        cnt += 1
        if cnt<=1000:
            continue
        if cnt>2000:
            break
            
            
        print('loading data {}/{} ...'.format(cnt, len(df_preprocess_pacu)), end='')
        row_demo = df_demograph[df_demograph['caseids']==row['caseids']].iloc[0]
            

        LEN = LEN_PER_NRS*SRATE
        # vital data - PPG (resampling 100 Hz to 250 Hz)
        df_vital = pickle.load(open(vital_path+row['file_path'], 'rb')).reset_index()
        pleth_samp = df_vital[['PPG']].fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        pleth_resamp = signal.resample(pleth_samp, LEN)
        #ppg_per_NRS = np.full(LEN, np.nan)
        #ppg_per_NRS[-len(pleth_resamp):] = pleth_resamp
        ppg_per_NRS = pleth_resamp
        

        # vital data - ECG (250Hz)
        ecg_samp = pickle.load(open(ecg_path+row['file_path'], 'rb')).reset_index()[['ECG']]
        ecg_samp = ecg_samp.fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        #ecg_per_NRS = np.full(LEN,np.nan)
        #ecg_per_NRS[-len(ecg_samp):] = ecg_samp
        ecg_per_NRS = ecg_samp
        
   
        if len(pleth_resamp) != LEN or len(ecg_samp) != LEN:
            print(row['caseids'])
            break
            

        save_path = '../../cranberry2/Preprocessing/cache/lowess_filtered/preprocess5/pacu_'+row['file_path']
        # 한 NRS에 대해 23개의 input 확인
        for i in range(23,57):
            # input이 전처리 통과한 경우
            if row[str(i+1)][0]:
                if os.path.exists(save_path+'_{}'.format(i)) and row['file_path'] not in vital_list:
                    ppg_inp, ecg_inp = pickle.load(open(save_path+'_{}'.format(i), 'rb'))
                
                else:
                    start_idx = i*OVERLAP*SRATE # 500i
                    end_idx = (i*OVERLAP+LEN_INPUT)*SRATE # 500i + 1000

                    # PPG smoothing
                    ppg_inp = np.copy(ppg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ppg_inp))!=0:
                        ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                    ppg_inp = ppg_inp - smooth(ppg_inp)
                    
                    # ECG smoothing
                    ecg_inp = np.copy(ecg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ecg_inp))!=0:
                        ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                    ecg_inp = ecg_inp - smooth(ecg_inp)                    
                
                    # pickle에 저장
                    pickle.dump([ppg_inp, ecg_inp],  open(save_path+'_{}'.format(i), 'wb'))
                    print('{}...'.format(i+1), end='')
        print('done')
'''                
                # normalization
                pleth_inp = ppg_inp - np.nanmean(ppg_inp)
                ecg_inp2 = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)
                
                
                # 해당 caseid가 test set에 속하는 경우
                if row['caseids'] in caseid_test:
                    age_test_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_test_pacu.append(1)
                    else:
                        gender_test_pacu.append(0)
                    x_test_pacu.append([pleth_inp, ecg_inp2])
                    y_test_pacu.append(int(float(row['NRS'])))

                # 해당 caseid가 train set에 해당하는 경우
                elif row['caseids'] in caseid_val:
                    age_val_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_val_pacu.append(1)
                    else:
                        gender_val_pacu.append(0)                    
                    x_val_pacu.append([pleth_inp, ecg_inp2])
                    y_val_pacu.append(int(float(row['NRS'])))
                    
                elif row['caseids'] in caseid_train:
                    age_train_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_train_pacu.append(1)
                    else:
                        gender_train_pacu.append(0)                    
                    x_train_pacu.append([pleth_inp, ecg_inp2])
                    y_train_pacu.append(int(float(row['NRS'])))
                    
                else:
                    non_lis.append(row['caseids'])
                    
        print('completed')
        
        
    x_train_pacu = np.array(x_train_pacu, np.float32)
    x_test_pacu = np.array(x_test_pacu, np.float32)
    y_train_pacu = np.array(y_train_pacu, int)
    y_test_pacu = np.array(y_test_pacu, int)
    x_val_pacu = np.array(x_val_pacu, np.float32)
    y_val_pacu = np.array(y_val_pacu, int)
    
    age_train_pacu = np.array(age_train_pacu, int)
    age_test_pacu = np.array(age_test_pacu, int)
    age_val_pacu = np.array(age_val_pacu, int)
    gender_train_pacu = np.array(gender_train_pacu, int)
    gender_test_pacu = np.array(gender_test_pacu, int)
    gender_val_pacu = np.array(gender_val_pacu, int)
    
        
    # 저장하기
    print('saving...', end='', flush=True)
    np.savez_compressed(input_path+'x_train_pacu2.npz', x_train_pacu)
    np.savez_compressed(input_path+'x_test_pacu2.npz', x_test_pacu)
    np.savez_compressed(input_path+'x_val_pacu2.npz', x_val_pacu)
    np.savez_compressed(input_path+'y_train_pacu2.npz', y_train_pacu)
    np.savez_compressed(input_path+'y_test_pacu2.npz', y_test_pacu)
    np.savez_compressed(input_path+'y_val_pacu2.npz', y_val_pacu)
    
    np.savez_compressed(input_path+'age_train_pacu2.npz', age_train_pacu)
    np.savez_compressed(input_path+'age_test_pacu2.npz', age_test_pacu)
    np.savez_compressed(input_path+'age_val_pacu2.npz', age_val_pacu)    
    np.savez_compressed(input_path+'gender_train_pacu2.npz', gender_train_pacu)
    np.savez_compressed(input_path+'gender_test_pacu2.npz', gender_test_pacu)
    np.savez_compressed(input_path+'gender_val_pacu2.npz', gender_val_pacu)    
    
    print('done', flush=True)

    
    
    
print('size of training set(pacu):', len(x_train_pacu))
print('size of validation set(pacu):', len(x_val_pacu))
print('size of test set(pacu):', len(x_test_pacu))
'''

loading data 1001/9949 ...done
loading data 1002/9949 ...done
loading data 1003/9949 ...done
loading data 1004/9949 ...done
loading data 1005/9949 ...done
loading data 1006/9949 ...done
loading data 1007/9949 ...done
loading data 1008/9949 ...done
loading data 1009/9949 ...done
loading data 1010/9949 ...done
loading data 1011/9949 ...done
loading data 1012/9949 ...29...done
loading data 1013/9949 ...done
loading data 1014/9949 ...done
loading data 1015/9949 ...done
loading data 1016/9949 ...done
loading data 1017/9949 ...done
loading data 1018/9949 ...done
loading data 1019/9949 ...done
loading data 1020/9949 ...done
loading data 1021/9949 ...done
loading data 1022/9949 ...done
loading data 1023/9949 ...done
loading data 1024/9949 ...done
loading data 1025/9949 ...done
loading data 1026/9949 ...done
loading data 1027/9949 ...done
loading data 1028/9949 ...done
loading data 1029/9949 ...done
loading data 1030/9949 ...done
loading data 1031/9949 ...done
loading data 1032/9949 ...done
loa

loading data 1243/9949 ...24...25...26...27...28...29...30...35...36...37...38...39...done
loading data 1244/9949 ...done
loading data 1245/9949 ...done
loading data 1246/9949 ...done
loading data 1247/9949 ...done
loading data 1248/9949 ...done
loading data 1249/9949 ...done
loading data 1250/9949 ...done
loading data 1251/9949 ...done
loading data 1252/9949 ...done
loading data 1253/9949 ...done
loading data 1254/9949 ...24...25...26...27...28...29...30...45...46...47...48...49...50...done
loading data 1255/9949 ...done
loading data 1256/9949 ...done
loading data 1257/9949 ...done
loading data 1258/9949 ...done
loading data 1259/9949 ...done
loading data 1260/9949 ...done
loading data 1261/9949 ...done
loading data 1262/9949 ...done
loading data 1263/9949 ...done
loading data 1264/9949 ...done
loading data 1265/9949 ...done
loading data 1266/9949 ...done
loading data 1267/9949 ...done
loading data 1268/9949 ...done
loading data 1269/9949 ...25...26...27...28...29...30...31...32...33.

loading data 1496/9949 ...done
loading data 1497/9949 ...done
loading data 1498/9949 ...done
loading data 1499/9949 ...done
loading data 1500/9949 ...29...done
loading data 1501/9949 ...done
loading data 1502/9949 ...done
loading data 1503/9949 ...done
loading data 1504/9949 ...done
loading data 1505/9949 ...done
loading data 1506/9949 ...done
loading data 1507/9949 ...done
loading data 1508/9949 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...done
loading data 1509/9949 ...done
loading data 1510/9949 ...done
loading data 1511/9949 ...done
loading data 1512/9949 ...done
loading data 1513/9949 ...done
loading data 1514/9949 ...done
loading data 1515/9949 ...done
loading data 1516/9949 ...done
loading data 1517/9949 ...done
loading data 1518/9949 ...done
loading data 1519/9949 ...done
loading data 1520/9949 ...done
loading data 1521/9949 ...done
loading data 1522/9949 ...done
loading data 1523/9949 ...24...34...35...36...done
loading data 1524/9949

loading data 1734/9949 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...done
loading data 1735/9949 ...done
loading data 1736/9949 ...done
loading data 1737/9949 ...done
loading data 1738/9949 ...done
loading data 1739/9949 ...done
loading data 1740/9949 ...done
loading data 1741/9949 ...done
loading data 1742/9949 ...done
loading data 1743/9949 ...done
loading data 1744/9949 ...done
loading data 1745/9949 ...done
loading data 1746/9949 ...done
loading data 1747/9949 ...done
loading data 1748/9949 ...24...25...26...27...28...29...30...31...32...33...34...35...36...done
loading data 1749/9949 ...done
loading data 1750/9949 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...done
loading data 1751/9949 ...done
loading data 1752/9949 ...done
loading data 1753/9949 ...done
loading data 1754/9949 ...done
load

loading data 1991/9949 ...35...36...37...43...44...45...46...48...done
loading data 1992/9949 ...done
loading data 1993/9949 ...done
loading data 1994/9949 ...done
loading data 1995/9949 ...done
loading data 1996/9949 ...done
loading data 1997/9949 ...done
loading data 1998/9949 ...done
loading data 1999/9949 ...done
loading data 2000/9949 ...done


"                \n                # normalization\n                pleth_inp = ppg_inp - np.nanmean(ppg_inp)\n                ecg_inp2 = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)\n                \n                \n                # 해당 caseid가 test set에 속하는 경우\n                if row['caseids'] in caseid_test:\n                    age_test_pacu.append(int(row_demo['age']))\n                    if row_demo['gender']=='F':\n                        gender_test_pacu.append(1)\n                    else:\n                        gender_test_pacu.append(0)\n                    x_test_pacu.append([pleth_inp, ecg_inp2])\n                    y_test_pacu.append(int(float(row['NRS'])))\n\n                # 해당 caseid가 train set에 해당하는 경우\n                elif row['caseids'] in caseid_val:\n                    age_val_pacu.append(int(row_demo['age']))\n                    if row_demo['gender']=='F':\n                        gender_val_pacu.append(1)\n                    else:\n          